In [1]:
import findspark

In [2]:
findspark.init('/home/nardao666/spark-2.4.0-bin-hadoop2.7/')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [10]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [11]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', 
                      predictionCol = 'prediction')

In [12]:
lrModel = lr.fit(training)

In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [14]:
lrModel.intercept

0.14228558260358093

In [15]:
training_summary = lrModel.summary

In [16]:
training_summary.r2

0.027839179518600154

In [17]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [20]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 332|
|   mean|-0.13443689390107816|
| stddev|  10.307948227647222|
|    min| -28.046018037776633|
|    max|  26.903524792043335|
+-------+--------------------+



In [23]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                169|
|   mean| 1.0256470336973225|
| stddev|  10.32468161462266|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [24]:
correct_model = lr.fit(train_data)

In [25]:
test_results = correct_model.evaluate(test_data)

In [27]:
test_results.rootMeanSquaredError

10.281732317738994

In [28]:
unlabel_data = test_data.select('features')

In [30]:
unlabel_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [31]:
predictions = correct_model.transform(unlabel_data)

In [33]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.6175015231083831|
|(10,[0,1,2,3,4,5,...| -0.8733198981503725|
|(10,[0,1,2,3,4,5,...|  1.1024595019065457|
|(10,[0,1,2,3,4,5,...|-0.00435692958752...|
|(10,[0,1,2,3,4,5,...|  -2.615112328797483|
|(10,[0,1,2,3,4,5,...| -0.1450470174907292|
|(10,[0,1,2,3,4,5,...| -1.3471910384154717|
|(10,[0,1,2,3,4,5,...|  -1.324201662217531|
|(10,[0,1,2,3,4,5,...|   1.014720601622181|
|(10,[0,1,2,3,4,5,...| 0.31522288213894956|
|(10,[0,1,2,3,4,5,...|  1.7854609809898245|
|(10,[0,1,2,3,4,5,...| -1.1544179388251385|
|(10,[0,1,2,3,4,5,...| -0.9220746068384459|
|(10,[0,1,2,3,4,5,...| 0.23986320801534558|
|(10,[0,1,2,3,4,5,...| -0.7733392235703362|
|(10,[0,1,2,3,4,5,...|  0.3727522261942352|
|(10,[0,1,2,3,4,5,...|   1.363853408818985|
|(10,[0,1,2,3,4,5,...| -1.2266297967034518|
|(10,[0,1,2,3,4,5,...| -1.2975285171221371|
|(10,[0,1,2,3,4,5,...|  1.795962